In [1]:
from langchain_community.document_loaders import TextLoader

loader =  TextLoader("Banking FAQS.txt")
data = loader.load()
#data

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total number of documents:", len(docs))

Total number of documents: 45


In [3]:
docs[4]

Document(metadata={'source': 'Banking FAQS.txt'}, page_content='Q: How Does A Customer Request A Cheque Book?\nA: The customer will need to complete a cheque book requisition slip which can be gotten from any branch. Otherwise, the customer can write to the bank requesting a chequebook. In both cases, the request will need to be signed according to the mandate of the account. He can also apply via the Internet Banking platform or by self-service.\nQ: How Long Does It Take For The Request To Be Processed?\nA: It takes 24 hours for Lagos branches and 48 ” 72 hours for up-country branches (branches outside Lagos),\nQ: How Many Leaves Are Contained In A Cheque Book And How Much Does It Cost?\nA: A cheque book for an individual account may contain 20 or 50 leaves, and for a corporate account 50 or 100 leaves. 20leaves cost ₦525, 50leaves cost ₦1050 and 100leaves cost ₦2100.\nQ: Can A Cheque Book Request Be Made In One Branch And The Cheque Book Collected From Another?')

In [9]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [10]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]
#vector

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

In [13]:
 vectorstore = Chroma.from_documents(documents=docs, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

In [16]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

retrieved_docs = retriever.invoke("What loans can i get")

In [18]:
len(retrieved_docs)

10

In [20]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", 
                             temperature=0.3, 
                             max_tokens=500)

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

system_prompt = (
    """
    You are a customer support chatbot for answering customer-related queries.
    Use the following pieces of retrieved context to answer the question.
    If the answers are not in the context, you can answer to the best of your ability
    but do not lie. If you do not know the answer, say that you do not know.
    Use a concise and friendly tone and be professional and helpful.
    """
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("system", "{context}"),  
        ("human", "{input}"),
    ]
)

In [33]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [35]:
response = rag_chain.invoke({"input": "How do i open an account"})
print(response["answer"])

To open a salary account, your company/organization will need to write a letter to the bank introducing you as a prospective account holder.

Alternatively, accounts can be opened with any amount or with Zero balance as there is no minimum account opening amount requirement for all account classes with the exception of our Gold and Platinum Accounts that have a N1 million and N3 million requirement respectively. Accounts are opened within 24 hours upon submission of complete documentation.
